# Requisites

- Environment with dependencies in `deploy_env.yaml`
- Downloaded UniRef database https://wwwuser.gwdguser.de/~compbiol/uniclust/2023_02

In [1]:
path = "predict" # Path to write files and results to

uniref_path = "/data/fnerin/UniRef30_2023_02/UniRef30_2023_02" # Path to the uncompressed UniRef database

# Functions

In [2]:
import sys
sys.path.append("training_data")

In [3]:
from utils.new_pdbs import Pdb
from utils.structure_fixing import get_fixed_structure, CifFileWriter
from utils.utils import Cif

In [4]:
import os, tempfile
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
os.makedirs(path, exist_ok=True)

Pdb.path = path
Pdb.original_cifs_path = path
Cif.path = path
Cif.original_cifs_path = path

In [6]:
def get_cif(
    pdb_id,
    path=path
):
    pdb = Pdb(pdb_id.lower())

    # Save original and uncompressed cif
    with open(f"{path}/{pdb.entry_id}_updated.cif.gz", "wb") as f:
        f.write(pdb.cif._cif_content)
    with open(f"{path}/{pdb.entry_id}_updated.cif", "w") as f:
        f.write(pdb.cif.text)
        
    # Cache the contents of the file
    pdb.cif.data
    return pdb

In [7]:
import pymol2

In [8]:
def get_site(site, only_protein=True, threshold=6): # site.pdb CAN BE PDB OR ASSEMBLY (must have .cif and .residues)
    """
    Function to, given a site, return a standardized list of residues from the parent structure that define the site with the Python interface of open-source PyMOL
    """    
    # Define the PyMOL-style selection of the modulator residues 
    sele = " or ".join(
        f"{res['label_asym_id']}/{res['auth_asym_id']}/{res['auth_comp_id']}`{res['auth_seq_id']}{res['pdbx_PDB_ins_code'].replace('?', '')}/*"
        for i, res in site.modulator_residues.iterrows()
    )
    
    with pymol2.PyMOL() as pymol:
        pymol.cmd.feedback(
            "disable", "executive", "details"
        )  # to silence "ExecutiveLoad-Detail: Detected mmCIF"

        # Load the parent structure of the site to PyMOL (it can only read a "real" file and not from string)
        with tempfile.NamedTemporaryFile("w+", suffix=".cif") as f:
            f.write(site.pdb.cif.text)
            pymol.cmd.load(f.name)

        # Retrieve all atoms within the threshold of the modulator selection
        site_atoms = pymol.cmd.get_model(f"br. all within {threshold} of {sele}")

    # Process the atom selection to obtain residue identifiers
    site_list = set(
        tuple(
            (
                a.segi, a.chain, a.resn,
                a.resi_number, a.ins_code or '?' # pdbx_PDB_ins_code or "?" if none
            ) 
            for a in site_atoms.atom
        )
    )

    # Transform the PyMOL-derived residue identifiers into a standard table of residues that can be used to retrieve the rows/residues from the parent structure's .residues table
    site_res = site.pdb.residues.merge(
        pd.DataFrame(
            site_list,
            columns=[
                "label_asym_id", "auth_asym_id", "auth_comp_id",
                "auth_seq_id", "pdbx_PDB_ins_code"
            ],
            dtype=str
        )
    ).query("pdbx_PDB_model_num == '1'")

    if only_protein:
        site_res = site_res.query(f"label_entity_id in {site.pdb._protein_entities} and label_asym_id not in {site.modulator_residues.label_asym_id.unique().tolist()}")

    assert len(site_res) > 0, "Site selection doesn't have any residues"

    return site_res

In [9]:
class Site:
    def __init__(self, pdb, modulator_residues=None, residues=None, only_protein=True, distance_threshold=6):
        self.pdb = pdb
        if modulator_residues is not None:
            self.modulator_residues = modulator_residues
            self.residues = get_site(self, only_protein=only_protein, threshold=distance_threshold)
        elif residues is not None:
            self.residues = pdb.residues.merge(pd.DataFrame(residues, dtype=str)).query("pdbx_PDB_model_num == '1'")
            if only_protein:
                self.residues = self.residues.query(f"label_entity_id in {site.pdb._protein_entities}")
        else:
            raise Exception("Pass one of 'modulator_residues' or 'residues'")

In [10]:
def get_clean_pdb(pdb, protein_chains, path=path):
    fixed_structure = get_fixed_structure(pdb, pdb, list(protein_chains), path, save=True)
    with open(f"{path}/{pdb.entry_id}.cif", "w+") as f:
        writer = CifFileWriter(f.name, compress=False)
        writer.write({
            pdb.entry_id.upper(): {
                "_atom_site": fixed_structure.to_dict(orient="list"),
                "_entity_poly": pdb.cif.data["_entity_poly"]
            }
        })
        
    cif = Cif(pdb.entry_id)
    # Cache the contents of the files
    cif.origcif.data
    cif.cif.data
    return cif

In [11]:
from ipymolstar import PDBeMolstar

In [12]:
def view_pdb(pdb, **kwargs):
    return PDBeMolstar(
        custom_data = {
                'data': pdb.cif.text,
                'format': 'cif',
                'binary': False,
            },
        sequence_panel = True,
        assembly_id='',
        **kwargs
    )

In [13]:
colors = {
    "orange": "#0FD55E00".lower(),
    "green": "#0F009E73".lower(),
    "blue": "#0F0072B2".lower()
}

def get_pocket(pdb, pocket, path=path):
    pocketn = pocket.replace('pocket', '')
    pocket_atoms = (
        Cif(pdb, f"{path}/{pdb}/{pdb}_out/{pdb}_out.cif", name=f"{pdb}_out")
        .atoms
        .query(f"label_comp_id == 'STP' and label_seq_id == '{pocketn}'")
    )
    pocket_atoms["label_asym_id"] = 'ZZZ'
    pocket_atoms["label_entity_id"] = '99'

    return pocket_atoms
    

def view_pockets(
    pdb,
    pockets:dict, # {"pocketn": {"color": ""}}
    protein_chains=None,
    site_residues=None,
    modulator_residues=None,
    path=path
):
    # Establish PDB
    if type(pdb) == str:
        pdb = Cif(pdb, filename=f"{path}/{pdb}_updated.cif")
        
    chains = protein_chains or pdb.residues.query(f"label_entity_id in {pdb._protein_entities}").label_asym_id.unique().tolist()
    pdb = pdb.entry_id
    cif = Cif(pdb, f"{path}/{pdb}_updated.cif")

    pockets = {
        pocketn: {
            "atoms": get_pocket(pdb, pocketn, path=path),
            "color": colors.get(pocket["color"], pocket["color"])
        }
        for pocketn, pocket in pockets.items()
    }

    # Fake entity data
    entities = pd.concat((
        pd.DataFrame(cif.cif.data["_entity"], dtype=str),#.query(f"id in {minimal_elements('label_entity_id')}"),
        pd.DataFrame([{"id": "99", "type": "branched", "pdbx_description": "pockets"}]) # Fake the pockets as carbohydrates to manage their representation
    )).fillna(".")

    columns = list( set.intersection( *map(set, (pocket["atoms"].columns for pocket in pockets.values())) ) )
    atoms = pd.concat((
        cif.atoms[columns], # if label_asym_id in protein_chains or modulator_residues.chains or label_entity_id not in protein_entities
        *(pocket["atoms"][columns] for pocket in pockets.values())
    ))

    with tempfile.NamedTemporaryFile("w+", suffix=".cif") as f:
        writer = CifFileWriter(f.name)
        writer.write({cif.entry_id.upper(): {
            "_entity": entities.to_dict(orient="list"),
            "_atom_site": atoms.to_dict(orient="list"),
        }})
        combined = Cif(pdb, filename=f.name)
        combined.cif.data # to cache it while 'f' exists

    data = [
        # Protein
        {"struct_asym_id": asym_id, 'representation': 'cartoon', 'representationColor': '#AEAEAE', 'focus': True}
        for asym_id in chains
    ]

    if site_residues is not None:
        data += [
            {'struct_asym_id': r["label_asym_id"], 'residue_number': int(r["label_seq_id"]), 'representationColor': colors["green"]}
            for i, r in site_residues.iterrows()
        ]
        
    # Ligands and molecules
    if modulator_residues is not None:
        data += [
            {'struct_asym_id': r["label_asym_id"], 'color': 'white'}
            for i, r in (
                combined.residues
                # Not modulator residues and only small molecule entities
                .merge(
                    modulator_residues if modulator_residues is not None else pd.DataFrame(columns=combined.residues.columns), # if modulator_residues not passed, empty df
                    how="outer", indicator=True
                )
                .query(f"""_merge == 'left_only' and label_entity_id in {entities.query("type == 'non-polymer'").id.unique().tolist()}""")
                .drop(columns="_merge")
                .iterrows()
            
            )
        ]

    # Pockets
    data += [
        {
            "struct_asym_id": "ZZZ", 'residue_number': int(pocketn.replace('pocket', '')), 'representation': 'point', 'representationColor': pocket["color"]
        }
        for pocketn, pocket in pockets.items()
    ]

    data += [
        {
            "struct_asym_id": "ZZZ", 'residue_number': int(pocketn.replace('pocket', '')), 'representation': 'gaussian-volume', 'representationColor': pocket["color"]
        }
        for pocketn, pocket in pockets.items()
    ]

    return view_pdb(
        combined,
        
        hide_polymer = True,
        # hide_heteroatoms = True,
        # hide_non_standard = True,
        hide_carbs = True,
        hide_water = True,
        
        color_data = {
            "data": data,
            "nonSelectedColor": None,
            "keepColors": True,
            "keepRepresentations": False,
        }
    )

In [14]:
from utils.pocket_utils import Pocket, get_pockets_info, get_mean_pocket_features
from utils.features_classes import * # Each FClass
from utils.features_utils import calculate_features, get_pdb_features

# Path to the mkdssp executable downloaded from https://github.com/PDB-REDO/dssp/releases/tag/v4.4.0
BiopythonF.dssp_path = "training_data/utils/external/mkdssp-4.4.0-linux-x64" 
os.chmod(BiopythonF.dssp_path, 0o755)
# f"mkdssp --mmcif-dictionary {os.environ['CONDA_PREFIX']}/share/libcifpp/mmcif_pdbx.dic"#"training_data/utils/external/mkdssp-4.4.0-linux-x64"
# Path to the UniRef (or other protein sequences database) downloaded from https://wwwuser.gwdguser.de/~compbiol/uniclust/2023_02
HHBlitsF.uniref_path = uniref_path

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python311.Release 2025.19+release.1354d05daa4c339d591afeecef3c94ca2d38680e 2025-05-07T12:36:04] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.ubuntu.cxx11thread.ser

In [15]:
from autogluon.tabular import TabularDataset, TabularPredictor

model = TabularPredictor.load("models/pockets_physchem_deploy")

In [16]:
def predict(
    pdb,
    protein_chains=None,
    path=path
):
    # Establish PDB
    if type(pdb) == str:
        pdb = Cif(pdb, filename=f"{path}/{pdb}_updated.cif")

    # Clean PDB
    protein_chains = protein_chains or pdb.residues.query(f"label_entity_id in {pdb._protein_entities}").label_asym_id.unique().tolist()

    clean_pdb = get_clean_pdb(
        pdb,
        protein_chains=protein_chains
    )

    
    # Pockets
    if not os.path.isdir(f"{path}/{clean_pdb.entry_id}/{clean_pdb.entry_id}_out"):
        os.makedirs(f"{path}/{clean_pdb.entry_id}", exist_ok=True)
        os.system(f"cp {clean_pdb.filename} {path}/{clean_pdb.entry_id}/")
        os.system(f"fpocket -m 3 -M 6 -i 35 --file {path}/{clean_pdb.entry_id}/{clean_pdb.entry_id}.cif")

    pockets = pd.DataFrame((
        {"pocket": (
            pocketf.split("_")[0]
            for pocketf in os.listdir(f"{path}/{clean_pdb.entry_id}/{clean_pdb.entry_id}_out/pockets")
                if pocketf.endswith(".cif")
        )}
    ))
    pockets["pdb"] = clean_pdb.entry_id
    
    pockets_features = pd.concat(
        (
            pockets,
            pockets.apply(
                lambda row: pd.Series(
                    Pocket(f"{path}/{clean_pdb.entry_id}/{clean_pdb.entry_id}_out/pockets/{row['pocket']}_atm.cif").feats
                ), axis=1
            )
        ),
        axis=1
    )

    pockets_features = pd.concat(
        (
            pockets_features[["pdb", "pocket"]],# [["pdb", "pocket", "nres", "site_in_pocket", "pocket_in_site"]],
            # pockets_features["label"],
            pockets_features.drop(columns=["pdb", "pocket"])#["pdb", "pocket", "nres", "site_in_pocket", "pocket_in_site"]) # "label", 
        ),
        axis=1,
        # keys=["Pockets", "Label", "FPocket"]
        keys=["Pockets", "FPocket"]
    )

    
    # Features
    os.makedirs(f"{path}/features/{clean_pdb.entry_id}", exist_ok=True)

    progressbar = tqdm(FClasses)
    for fc in progressbar:
        progressbar.set_description(f"Calculating {fc.__name__[:-1]}")
        file = f"{path}/features/{clean_pdb.entry_id}/{fc.__name__}.pkl"
        if not os.path.isfile(file):
            calculated = calculate_features(clean_pdb.entry_id, fc, file, path, path)
            assert calculated, f"Feature calculation failed: {fc}"
            
    features = get_pdb_features(
        clean_pdb,
        sites = [pd.DataFrame(columns=clean_pdb.residues.columns),],
        features_path = path
    )

    
    # Data and prediction
    df = pd.concat(
        (
            pockets_features,
            pockets_features.apply(
                lambda row: get_mean_pocket_features(
                    clean_pdb.entry_id, #row[("Pockets", "pdb")],
                    row[("Pockets", "pocket")],
                    pdb_features = features,
                    pockets_path = path # # f"{pockets_path}/{pdb}/{pdb}_out/pockets/{pocket}_atm.cif"
                ), 
                axis=1 
            )
        ),
        axis=1
    )

    df.index = df["Pockets"][["pdb", "pocket"]].apply(lambda x: "_".join(x), axis=1)
    df = df.drop(columns=["Pockets"], level=0)
    df.columns = map(lambda x: "_".join(x), df.columns.values)
    
    preds = model.predict_proba(TabularDataset(df))[[1]].sort_values(1, ascending=False).rename(columns={1: "Allosteric score"})
    preds.index = preds.index.map(lambda x: x.split("_")[-1])
    return preds

# Predict

## Protein structure

In [17]:
# 2xr6, 7nl6

In [18]:
pdb_id = "4or2"

In [19]:
pdb = get_cif(
    pdb_id
)

view_pdb(pdb)

PDBeMolstar(bg_color='#F7F7F7', custom_data={'data': "data_4OR2\n#\n_entry.id 4OR2\n#\n_citation.id primary\n_…

## Predict

In [20]:
predictions = predict(
    pdb_id,
    protein_chains=["A"]
)

  0%|          | 0/9 [00:00<?, ?it/s]

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.320535 seconds.
core.import_pose.import_pose: File 'predict/4or2.cif' automatically determined to be of type mmCIF from contents.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLU:CtermProteinFull 360
core.conformation.Conformation: Found disulfide between residues 182 263


2025-05-16 15:15:55,228 - .prody - WARNING - WARNING Could not find _atom_site_anisotrop in lines.
2025-05-16 15:15:55,229 - .prody - WARNING - WARNING No anisotropic B factors found
2025-05-16 15:15:55,229 - .prody - DEBUG - 2817 atoms and 1 coordinate set(s) were parsed in 0.02s.
2025-05-16 15:15:55,280 - .prody - DEBUG - Hessian was built in 0.05s.
2025-05-16 15:15:56,775 - .prody - DEBUG - 1074 modes were calculated in 1.50s.
2025-05-16 15:15:56,820 - .prody - INFO - Calculating stiffness matrix.
2025-05-16 15:15:57,020 - .prody - DEBUG - Stiffness matrix calculated in 0.20s.
2025-05-16 15:15:57,021 - .prody - INFO - The range of effective force constant is: 4.607130302470655 to 26.14793112720262.
2025-05-16 15:15:57,031 - .prody - DEBUG - Kirchhoff was built in 0.01s.
2025-05-16 15:15:57,035 - .prody - DEBUG - 10 modes were calculated in 0.00s.
2025-05-16 15:15:57,045 - .prody - DEBUG - Kirchhoff was built in 0.01s.
2025-05-16 15:15:57,050 - .prody - DEBUG - 10 modes were calculat

In [21]:
predictions

,Allosteric score
pocket1,0.920452
pocket12,0.432812
pocket2,0.029577
pocket4,0.004039
pocket9,0.003933
pocket11,0.002201
pocket5,0.001827
pocket3,0.000979
pocket10,0.000667
pocket6,0.000090


## View

In [22]:
view_pockets(
    pdb_id,
    pockets={"pocket1": {"color": "green"}, "pocket12": {"color": "blue"}}, # {"pocketn": {"color": ""}}
    protein_chains=["A"]
)

PDBeMolstar(bg_color='#F7F7F7', color_data={'data': [{'struct_asym_id': 'A', 'representation': 'cartoon', 'rep…

### Optional: view a target site

Set an object of the `Site` class passing a modulator molecule (and defining the site around it) or a list of residues. Then visualize it in the structure to assess overlap with predicted pockets (residues of the site will be colored green in the protein cartoon visualization).

<br>

#### With a modulator molecule

In [23]:
# Desired modulator is label_asym_id 'C'
pdb.residues.query("label_asym_id == 'C'")

,label_comp_id,label_asym_id,label_entity_id,label_seq_id,pdbx_PDB_ins_code,auth_seq_id,auth_comp_id,auth_asym_id,pdbx_PDB_model_num,pdbx_label_index,pdbx_sifts_xref_db_name,pdbx_sifts_xref_db_acc,pdbx_sifts_xref_db_num,pdbx_sifts_xref_db_res
5603,FM9,C,2,.,?,1901,FM9,A,1,1901,?,?,?,?


In [24]:
site = Site(
    pdb, 
    modulator_residues=pdb.residues.query("label_asym_id == 'C'")
)
site

In [25]:
view_pockets(
    pdb_id,
    pockets={"pocket1": {"color": "green"}, "pocket12": {"color": "blue"}}, # {"pocketn": {"color": ""}}
    protein_chains=["A"],
    site_residues=site.residues,
    modulator_residues=site.modulator_residues,
)

PDBeMolstar(bg_color='#F7F7F7', color_data={'data': [{'struct_asym_id': 'A', 'representation': 'cartoon', 'rep…

#### With a list of residues

In [26]:
# List of residue numbers of site
resnums = site.residues.label_seq_id.to_list()
resnums

['177',
 '189',
 '190',
 '193',
 '194',
 '197',
 '201',
 '246',
 '275',
 '277',
 '281',
 '282',
 '285',
 '286',
 '287',
 '289',
 '290',
 '293',
 '322',
 '323',
 '324',
 '326',
 '327',
 '328',
 '330',
 '334',
 '340',
 '341',
 '344',
 '347',
 '348',
 '351']

In [27]:
# Site can be defined with a list of residues instead of a modulator
res_site = Site(
    pdb=pdb,
    residues=[{"label_asym_id": "A", "label_seq_id": seqnum} for seqnum in resnums]
)
res_site.residues

,label_comp_id,label_asym_id,label_entity_id,label_seq_id,pdbx_PDB_ins_code,auth_seq_id,auth_comp_id,auth_asym_id,pdbx_PDB_model_num,pdbx_label_index,pdbx_sifts_xref_db_name,pdbx_sifts_xref_db_acc,pdbx_sifts_xref_db_num,pdbx_sifts_xref_db_res
0,LEU,A,1,177,?,648,LEU,A,1,177,UNP,Q13255,648,L
1,GLN,A,1,189,?,660,GLN,A,1,189,UNP,Q13255,660,Q
2,ARG,A,1,190,?,661,ARG,A,1,190,UNP,Q13255,661,R
3,VAL,A,1,193,?,664,VAL,A,1,193,UNP,Q13255,664,V
4,GLY,A,1,194,?,665,GLY,A,1,194,UNP,Q13255,665,G
5,SER,A,1,197,?,668,SER,A,1,197,UNP,Q13255,668,S
6,TYR,A,1,201,?,672,TYR,A,1,201,UNP,Q13255,672,Y
7,GLN,A,1,246,?,717,GLN,A,1,246,UNP,Q13255,717,Q
8,CYS,A,1,275,?,746,CYS,A,1,275,UNP,Q13255,746,C
9,THR,A,1,277,?,748,THR,A,1,277,UNP,Q13255,748,T


In [28]:
view_pockets(
    pdb_id,
    pockets={"pocket1": {"color": "green"}, "pocket12": {"color": "blue"}}, # {"pocketn": {"color": ""}}
    protein_chains=["A"],
    site_residues=res_site.residues,
)

PDBeMolstar(bg_color='#F7F7F7', color_data={'data': [{'struct_asym_id': 'A', 'representation': 'cartoon', 'rep…